<a href="https://colab.research.google.com/github/Purvanshsingh/LongFormer-Implementation/blob/master/LongFormer_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install pytorch-transformers
!pip install transformers==3.0.2

     |████████████████████████████████| 184kB 5.1MB/s 
     |████████████████████████████████| 1.1MB 18.7MB/s 
     |████████████████████████████████| 890kB 24.2MB/s 
     |████████████████████████████████| 102kB 8.4MB/s 
     |████████████████████████████████| 7.0MB 27.8MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=668bf7d82815cc03bcc3e2f89a7922576d0882ebabca641b60bc0f2a98ca1b6d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for boto3: filename=boto3-1.16.33-py2.py3-none-any.whl size=128451 sha256=96637f7202cbf18312d4ecc6dee9a352eeba829dad1351b74b6355cf71c9b483
  Stored in directory: /root/.cache/pip/wheels/28/c0/ba/5f9cce3a8686eea945fd594ae8fcdec24b1bd7402d3d065a47
Successfully built sacremoses boto3
ERROR: botocore 1.19.33 has requirement urllib3<1.27,>=1.25.4; python_version != "3.4", but you'll hav

In [4]:
pip install git+https://github.com/allenai/longformer.git

  Cloning https://github.com/allenai/longformer.git to /tmp/pip-req-build-qwndci90
  Running command git clone -q https://github.com/allenai/longformer.git /tmp/pip-req-build-qwndci90
  Cloning http://github.com/ibeltagy/pytorch-lightning.git (to revision v0.8.5_fixes) to /tmp/pip-install-d8c51axp/pytorch-lightning
  Running command git clone -q http://github.com/ibeltagy/pytorch-lightning.git /tmp/pip-install-d8c51axp/pytorch-lightning
  Running command git checkout -b v0.8.5_fixes --track origin/v0.8.5_fixes
  Switched to a new branch 'v0.8.5_fixes'
  Branch 'v0.8.5_fixes' set up to track remote branch 'v0.8.5_fixes' from 'origin'.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 748.8MB 24kB/s 
     |████████████████████████████████| 317kB 15.9MB/s 
     |████████████████████████████████| 1.7MB 46.0MB/s 
     |████████████████████████████████| 829kB 40.6MB/s 
     |██

In [5]:
!pip install datasets

     |████████████████████████████████| 163kB 5.0MB/s 


In [6]:
import pandas as pd
from transformers import LongformerTokenizerFast, LongformerForSequenceClassification, Trainer, TrainingArguments, LongformerConfig
import datasets
import torch.nn as nn
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import os

In [7]:
config = LongformerConfig()
print(config)
train_data, test_data = datasets.load_dataset('imdb', split =['train', 'test'], 
                                             cache_dir='/media/data_files/github/website_tutorials/data')

LongformerConfig {
  "attention_probs_dropout_prob": 0.1,
  "attention_window": 512,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



Dataset imdb downloaded and prepared to /media/data_files/github/website_tutorials/data/imdb/plain_text/1.0.0/90099cb476936b753383ba2ae6ab2eae419b2e87f71cd5189cb9c8e5814d12a3. Subsequent calls will reuse this data.


In [8]:
# load model and tokenizer and define length of the text sequence
model = LongformerForSequenceClassification.from_pretrained('allenai/longformer-base-4096',
                                                           gradient_checkpointing=False,
                                                           attention_window = 512)
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-base-4096', max_length = 1024)

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', '

In [9]:
model.config

LongformerConfig {
  "attention_mode": "longformer",
  "attention_probs_dropout_prob": 0.1,
  "attention_window": [
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "ignore_attention_mask": false,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 4098,
  "model_type": "longformer",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "sep_token_id": 2,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

In [10]:
# define a function that will tokenize the model, and will return the relevant inputs for the model
def tokenization(batched_text):
    return tokenizer(batched_text['text'], padding = 'max_length', truncation=True, max_length = 1024)

train_data = train_data.map(tokenization, batched = True, batch_size = len(train_data))
test_data = test_data.map(tokenization, batched = True, batch_size = len(test_data))

In [11]:
len(train_data['input_ids'][0])

1024

In [12]:
# define the training arguments
training_args = TrainingArguments(
    output_dir = '/media/data_files/github/website_tutorials/results',
    num_train_epochs = 5,
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 8,    
    per_device_eval_batch_size= 16,
    #disable_tqdm = False, 
    #load_best_model_at_end=True,
    warmup_steps=200,
    weight_decay=0.01,
    logging_steps = 4,
    fp16 = True,
    logging_dir='/media/data_files/github/website_tutorials/logs',
    #dataloader_num_workers = 0,
    #run_name = 'longformer-classification-updated-rtx3090_paper_replication_2_warm'
)

In [13]:
pwd

'/content'

In [37]:
# save the best model
#Trainer.is_world_master()
Trainer(args=training_args,model=model)._save('/content/')